In [32]:
import random
import shutil
import pandas as pd
import os
import sys
import numpy as np
from tqdm.notebook import tqdm
from skimage.io import imread
from skimage.exposure import match_histograms
import errno
import argparse
from tqdm import tqdm

In [27]:
def organizar_datos_clasificacion(image_dir, csv, out_dir):
    """
    Script modificado de Biapy para este dataset

    Parametros:
        image_dir (str): Ruta del directorio de imagenes.
        csv (str): Ruta del archivo CSV que contiene la clase de cada imagen.
        out_dir (str): Directorio de salida para organizar las imagenes.

    Ejemplo:
        organizar_datos_clasificacion('/ruta/del/directorio_imagenes', '/ruta/del/archivo.csv', '/ruta/del/directorio_salida')
    """
    df = pd.read_csv(csv)

    if 'filename' not in df.columns or 'label' not in df.columns:
        raise ValueError("El archivo CSV debe contener al menos dos columnas: 'filename' y 'label'")

    for index, row in tqdm(df.iterrows(), desc="Copiando imágenes"):
        dest_folder = os.path.join(out_dir, str(row['label']).replace(" ", "_").lower())
        os.makedirs(dest_folder, exist_ok=True)

        filename = os.path.join(image_dir, row['filename'])
        print("Copiando archivo {} a {}".format(filename, dest_folder))
        shutil.copy(filename, dest_folder)

In [2]:
def contar_imagenes_en_carpeta(carpeta):
    """
    Cuenta la cantidad de imágenes válidas en una carpeta.

    Esta función cuenta la cantidad de archivos de imagen válidos con extensiones '.jpg', '.png',
    '.jpeg' o '.gif' que se encuentran en una carpeta.

    Args:
        carpeta (str): Ruta de la carpeta que se desea contar.

    Returns:
        int: Cantidad de imágenes válidas encontradas en la carpeta.

    Example:
        contar_imagenes_en_carpeta("/ruta/carpeta/")
    """

    contador = 0
    extensiones_validas = ['.jpg', '.png', '.jpeg', '.gif']

    for archivo in os.listdir(carpeta):
        _, extension = os.path.splitext(archivo)
        if extension.lower() in extensiones_validas:
            contador += 1

    return contador


In [3]:

def eliminar_imagenes_excedentes(carpeta, objetivo):
    """
    Elimina imágenes excedentes de una carpeta para alcanzar un objetivo específico.

    Esta función cuenta la cantidad actual de imágenes en una carpeta y calcula la cantidad
    de imágenes excedentes que debe eliminar para alcanzar un objetivo específico de imágenes.

    Args:
        carpeta (str): Ruta de la carpeta que contiene las imágenes.
        objetivo (int): Número objetivo de imágenes que se desea mantener en la carpeta.

    Returns:
        None: La función no devuelve ningún valor, simplemente elimina imágenes excedentes.

    Example:
        eliminar_imagenes_excedentes("/ruta/carpeta", 1000)
    """

    # Obtiene la cantidad actual de imágenes en la carpeta
    imagenes_actuales = contar_imagenes_en_carpeta(carpeta)

    # Calcula la cantidad de imágenes excedentes a eliminar
    excedentes = imagenes_actuales - objetivo

    # Si no hay imágenes excedentes, no es necesario hacer cambios
    if excedentes <= 0:
        return

    # Obtiene una lista de imágenes excedentes de manera aleatoria
    imagenes_a_eliminar = random.sample(os.listdir(carpeta), excedentes)

    # Elimina las imágenes excedentes
    for imagen in imagenes_a_eliminar:
        ruta_imagen = os.path.join(carpeta, imagen)
        os.remove(ruta_imagen)


In [26]:
def mover_carpeta(origen, destino):
    """
    Mueve una carpeta desde la ruta de origen a la ruta de destino.

    Si la carpeta de destino no existe, se crea para permitir el movimiento.

    Parámetros:
        origen (str): Ruta de la carpeta de origen que se va a mover.
        destino (str): Ruta de la carpeta de destino.

    Ejemplo:
        mover_carpeta('/ruta/de/origen/carpeta', '/ruta/de/destino/carpeta')
    """
    if not os.path.exists(destino):
        os.makedirs(destino)

    shutil.move(origen, destino)


In [4]:
def organizar_csv(ruta_csv, ruta_img, ruta_nueva):
    """
    Organiza imágenes basadas en la información de un archivo CSV.

    Esta función lee un archivo CSV que contiene información sobre las imágenes y
    las organiza en subdirectorios según las etiquetas proporcionadas en el CSV.

    Args:
        ruta_csv (str): Ruta del archivo CSV que contiene la información de las imágenes.
        ruta_img (str): Ruta del directorio donde se encuentran las imágenes originales.
        ruta_nueva (str): Ruta del directorio donde se almacenarán las imágenes organizadas.

    Returns:
        None: La función no devuelve ningún valor, simplemente organiza las imágenes en el disco.

    Example:
        organizar_csv("datos.csv", "/ruta/imagenes/", "/ruta/imagenes_organizadas/")
    """

    # Carga los datos del csv en un DataFrame
    df = pd.read_csv(ruta_csv)

    # Ubicación original de las imágenes
    original_dir = ruta_img

    # Directorio donde se almacenarán las imágenes organizadas
    new_dir = ruta_nueva

    # Crea el directorio si no existe
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

    # Obtiene todas las etiquetas únicas y las ordena
    labels = sorted(df['label'].unique())

    # Diccionario para almacenar el mapeo de las etiquetas a los números enteros
    label_dict = {label: i for i, label in enumerate(labels)}

    # Itera sobre todas las filas del DataFrame
    for index, row in df.iterrows():
        # Consigue la etiqueta y el nombre del archivo
        label = row['label']
        filename = row['filename']

        # Crea un nuevo directorio para la etiqueta si no existe
        label_dir = os.path.join(new_dir, str(label_dict[label]))
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)

        # Copia la imagen al nuevo directorio
        original_file_path = os.path.join(original_dir, filename)
        new_file_path = os.path.join(label_dir, f'{index}.jpg')
        shutil.copyfile(original_file_path, new_file_path)


# **2D Classification pipeline**
___  
  
In this notebook we show how to apply a [BiaPy](https://biapy.readthedocs.io/en/latest/) pipeline for **2D classification** of butterfly data.

**Without any coding**, we explain step by step how to
1. **upload a set of training and test images** which need to be organized in folders, one for each class,
2. **train a deep neural network (DNN)** model on the training set,
3. **apply the model** to the test images, and
4. **download the classification results** to your local machine.

**Disclaimer:** The structure of the notebook is heavily inspired in the fantastic [ZeroCostDL4Mic notebooks](https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki).

**Contact:** This notebook has been made by [Ignacio Arganda-Carreras](mailto:ignacio.arganda@ehu.eus) and [Daniel Franco-Barranco](mailto:daniel.franco@dipc.org). If you have any suggestion or comment, or find any problem, please write us an email or [create an issue in BiaPy's repository](https://github.com/danifranco/BiaPy/issues). Thanks!

## **Expected inputs and outputs**
___
**Inputs**

This notebook expects three folders as input:
* **Training raw images**: with the raw 2D images to train the model.
* **Test raw images**: with the raw 2D images to test the model.
* **Output folder**: a path to store the classification results.

**Outputs**

If the execution is successful, a folder will be created containing the classification results. The resulting csv file can be downloaded at the end of the notebook.



## **Prepare the environment**
___

Establish connection with Google services. You **must be logged in to Google** to continue.
Since this is not Google's own code, you will probably see a message warning you of the dangers of running unfamiliar code. This is completely normal.

### **Download an example dataset**
---
If you do not have data at hand but would like to test the notebook, no worries! You can run the following cell to download an example dataset.

In particular, we will use a particular subset of [Butterfly](https://www.kaggle.com/datasets/phucthaiv02/butterfly-image-classification) dataset, concretely DermaMNIST dataset publicly available online.

In [5]:
os.chdir('/content/')
#https://drive.google.com/file/d//view?usp=sharing
!curl -L -s -o archive.zip 'https://drive.google.com/uc?id=1GBsn30dCC_XjGwtFLyMMhnAwTLFzW3vV&confirm=t'
!unzip -q archive.zip
!rm archive.zip
print('Dataset downloaded and unzipped under /content')

Dataset downloaded and unzipped under /content


## **Install BiaPy library**

In [6]:
#@markdown ##Play to install BiaPy and its dependences

# Clone the repo
os.chdir('/content/')
if not os.path.exists('BiaPy'):
    !git clone --depth 1 https://github.com/danifranco/BiaPy.git
    !pip install --upgrade --no-cache-dir gdown &> /dev/null
    sys.path.insert(0, 'BiaPy')
    os.chdir('/content/BiaPy')

    # Install dependencies
    !pip install git+https://github.com/aleju/imgaug.git &> /dev/null
    !pip install numpy_indexed yacs fill_voids edt &> /dev/null
else:
    print( 'Using existing installed version of BiaPy' )

Cloning into 'BiaPy'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 157 (delta 43), reused 83 (delta 19), pack-reused 0
Receiving objects: 100% (157/157), 26.38 MiB | 21.90 MiB/s, done.
Resolving deltas: 100% (43/43), done.



## **Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

Go to **Runtime -> Change the Runtime type**

**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

**Accelerator: GPU** *(Graphics processing unit)*

## **Paths to load input images and save output files**
___

If option 1 (uploading the folder) or option 3 (downloading our prepared data samples) were chosen, define train_data_path as '/content/data/train', val_data_path as '/content/data/val' (if not using validation from train which can be ignored if so), test_data_path as '/content/data/test' and output_path as '/content/out'. Please make sure you download the results from the '/content/out' folder later!

If option 2 is chosen, introduce here the paths to your input files and to the folder where you want to store the results. E.g. '/content/gdrive/MyDrive/...'.

In case you have troubles finding the path to your folders, at the top left of this notebook you will find a small folder icon. Explore until you find the folders. There you can copy the folder path by right clicking and clicking "copy".

MODO ANTIGUO, AHORA SE USA LA FUNCION **organizar_datos_clasificacion(...)** MAS ABAJO SE EJECUTA CORRECTAMENTE DE LA NUEVA MANERA, OMITIR ESTA EJECUCION


In [8]:
!python /content/BiaPy/utils/scripts/from_class_csv_to_folders.py --image_dir "/content/train" --csv "/content/Training_set.csv" --out_dir "/content/data/train"


Se han truncado las últimas 5000 líneas del flujo de salida.
Copying file /content/train/Image_1501.jpg to /content/data/train/copper_tail
Copying file /content/train/Image_1502.jpg to /content/data/train/danaid_eggfly
Copying file /content/train/Image_1503.jpg to /content/data/train/paper_kite
Copying file /content/train/Image_1504.jpg to /content/data/train/eastern_dapple_white
Copying file /content/train/Image_1505.jpg to /content/data/train/pipevine_swallow
Copying file /content/train/Image_1506.jpg to /content/data/train/purple_hairstreak
Copying file /content/train/Image_1507.jpg to /content/data/train/pipevine_swallow
Copying file /content/train/Image_1508.jpg to /content/data/train/scarce_swallow
Copying file /content/train/Image_1509.jpg to /content/data/train/danaid_eggfly
Copying file /content/train/Image_1510.jpg to /content/data/train/grey_hairstreak
Copying file /content/train/Image_1511.jpg to /content/data/train/american_snoot
Copying file /content/train/Image_1512.jpg 

NUEVA FORMA

In [28]:
# Rutas de los directorios y archivo CSV
image_dir = '/content/train'
csv = '/content/Training_set.csv'
out_dir = '/content/data/train'

In [29]:
organizar_datos_clasificacion(image_dir, csv, out_dir)

Copiando imágenes: 0it [00:00, ?it/s]

Se han truncado las últimas 5000 líneas del flujo de salida.
Copiando archivo /content/train/Image_1500.jpg a /content/data/train/black_hairstreak
Copiando archivo /content/train/Image_1501.jpg a /content/data/train/copper_tail
Copiando archivo /content/train/Image_1502.jpg a /content/data/train/danaid_eggfly
Copiando archivo /content/train/Image_1503.jpg a /content/data/train/paper_kite
Copiando archivo /content/train/Image_1504.jpg a /content/data/train/eastern_dapple_white
Copiando archivo /content/train/Image_1505.jpg a /content/data/train/pipevine_swallow
Copiando archivo /content/train/Image_1506.jpg a /content/data/train/purple_hairstreak
Copiando archivo /content/train/Image_1507.jpg a /content/data/train/pipevine_swallow
Copiando archivo /content/train/Image_1508.jpg a /content/data/train/scarce_swallow
Copiando archivo /content/train/Image_1509.jpg a /content/data/train/danaid_eggfly
Copiando archivo /content/train/Image_1510.jpg a /content/data/train/grey_hairstreak
Copiando

MOVER AUTOMATICAMENTE LA CARPETA TEST A DATA


In [25]:
origen = '/content/test'
destino = '/content/data/t_test'
mover_carpeta(origen, destino)

In [ ]:
directorio = "/content/data/train"

for carpeta in os.listdir(directorio):
    ruta_carpeta = os.path.join(directorio, carpeta)
    if os.path.isdir(ruta_carpeta):
        cantidad_imagenes = contar_imagenes_en_carpeta(ruta_carpeta)
        print(f"En la carpeta '{carpeta}' hay {cantidad_imagenes} imágenes.")

En la carpeta 'question_mark' hay 77 imágenes.
En la carpeta 'an_88' hay 85 imágenes.
En la carpeta 'crecent' hay 97 imágenes.
En la carpeta 'painted_lady' hay 78 imágenes.
En la carpeta 'tropical_leafwing' hay 83 imágenes.
En la carpeta 'indra_swallow' hay 81 imágenes.
En la carpeta 'black_hairstreak' hay 85 imágenes.
En la carpeta 'appollo' hay 90 imágenes.
En la carpeta 'silver_spot_skipper' hay 83 imágenes.
En la carpeta 'ulyses' hay 84 imágenes.
En la carpeta 'pipevine_swallow' hay 84 imágenes.
En la carpeta 'pine_white' hay 86 imágenes.
En la carpeta 'american_snoot' hay 74 imágenes.
En la carpeta 'straited_queen' hay 87 imágenes.
En la carpeta 'great_jay' hay 94 imágenes.
En la carpeta 'grey_hairstreak' hay 86 imágenes.
En la carpeta 'red_postman' hay 89 imágenes.
En la carpeta 'sleepy_orange' hay 107 imágenes.
En la carpeta 'clodius_parnassian' hay 87 imágenes.
En la carpeta 'large_marble' hay 81 imágenes.
En la carpeta 'julia' hay 81 imágenes.
En la carpeta 'gold_banded' hay 7

In [11]:
directorio = "/content/data/train"
objetivo_total_imagenes = 2500
objetivo_por_carpeta = objetivo_total_imagenes // len(os.listdir(directorio))

for carpeta in os.listdir(directorio):
    ruta_carpeta = os.path.join(directorio, carpeta)
    if os.path.isdir(ruta_carpeta):
        eliminar_imagenes_excedentes(ruta_carpeta, objetivo_por_carpeta)

cantidad_total_imagenes = sum(contar_imagenes_en_carpeta(os.path.join(directorio, carpeta)) for carpeta in os.listdir(directorio))
print(f"Se ha logrado un total de aproximadamente {cantidad_total_imagenes} imágenes.")


Se ha logrado un total de aproximadamente 2475 imágenes.


In [12]:
for carpeta in os.listdir(directorio):
    ruta_carpeta = os.path.join(directorio, carpeta)
    if os.path.isdir(ruta_carpeta):
        cantidad_imagenes = contar_imagenes_en_carpeta(ruta_carpeta)
        print(f"En la carpeta '{carpeta}' hay {cantidad_imagenes} imágenes.")

En la carpeta 'mourning_cloak' hay 33 imágenes.
En la carpeta 'paper_kite' hay 33 imágenes.
En la carpeta 'mangrove_skipper' hay 33 imágenes.
En la carpeta 'crecent' hay 33 imágenes.
En la carpeta 'red_postman' hay 33 imágenes.
En la carpeta 'banded_peacock' hay 33 imágenes.
En la carpeta 'yellow_swallow_tail' hay 33 imágenes.
En la carpeta 'pine_white' hay 33 imágenes.
En la carpeta 'pipevine_swallow' hay 33 imágenes.
En la carpeta 'purple_hairstreak' hay 33 imágenes.
En la carpeta 'chestnut' hay 33 imágenes.
En la carpeta 'sleepy_orange' hay 33 imágenes.
En la carpeta 'viceroy' hay 33 imágenes.
En la carpeta 'peacock' hay 33 imágenes.
En la carpeta 'question_mark' hay 33 imágenes.
En la carpeta 'painted_lady' hay 33 imágenes.
En la carpeta 'two_barred_flasher' hay 33 imágenes.
En la carpeta 'elbowed_pierrot' hay 33 imágenes.
En la carpeta 'copper_tail' hay 33 imágenes.
En la carpeta 'checquered_skipper' hay 33 imágenes.
En la carpeta 'blue_spotted_crow' hay 33 imágenes.
En la carpeta

In [13]:
directorio = "/content/data/train"
cantidad_carpetas = sum(1 for carpeta in os.listdir(directorio) if os.path.isdir(os.path.join(directorio, carpeta)))

print(f"El directorio '{directorio}' contiene {cantidad_carpetas} carpetas.")


El directorio '/content/data/train' contiene 75 carpetas.


In [ ]:
#organizar_csv('/content/Training_set.csv','/content/train', '/content/data/train')

In [14]:
#@markdown #####Path to train images
train_data_path = '/content/data/train' #@param {type:"string"}
#@markdown #####Path to validation images (necessary only if you do not want to extract validation from train, i.e. when **validation_from_train** variable below is **False**)
val_data_path = '' #@param {type:"string"}
#@markdown #####Path to test images
test_data_path = '/content/data/t_test' #@param {type:"string"}
#@markdown #####Path to store the resulting images (it'll be created if not existing):
output_path = '/content/output' #@param {type:"string"}

## **Configure and train the DNN model**
[BiaPy](https://biapy.readthedocs.io/en/latest/) contains a few deep learning models to perform classification.

The selection of the model and the pipeline hyperparameters can be configured by editing the YAML configuration file or (easier) by running the next cell.

### **Select your parameters**
---
#### **Name of the model**
* **`model_name`:** Use only my_model -style, not my-model (Use "_" not "-"). Do not use spaces in the name. Avoid using the name of an existing model (saved in the same folder) as it will be overwritten.

#### **Data management**

* **`validation_from_train`:** Select to extract validation data from the training samples. If is not selected the validation data path must be set in **val_data_path** variable above.

* **`percentage_validation`:**  Input the percentage of your training dataset you want to use to validate the network during the training. **Default value: 10**

* **`test_ground_truth`:** Select to use test data folder order as the ground truth class to measure the performance of the model's result. **Default value: True**

#### **Basic training parameters**
* **`number_of_classes`:** Input number of classes present in the problem. It must be equal to the number of subfolders in training and validation (if not extracted from train) folders.

* **`number_of_epochs`:** Input how many epochs (rounds) the network will be trained. For the example dataset, reasonable results can already be observed after 100 epochs. **Default value: 100**

* **`patience`:**  Input how many epochs you want to wait without the model improving its results in the validation set to stop training. **Default value: 20**

#### **Advanced Parameters - experienced users only**
* **`model_architecture`:**  Select the architecture of the DNN used as backbone of the pipeline. Options: EfficientNet B0 and a simple CNN. **Default value: EfficientNet B0**

* **`batch_size:`** This parameter defines the number of patches seen in each training step. Reducing or increasing the **batch size** may slow or speed up your training, respectively, and can influence network performance. **Default value: 12**

* **`patch_size`:** Input the size of the patches use to train your model (length in pixels in X and Y). The value should be smaller or equal to the dimensions of the image. **Default value: 28**

* **`input_channels`:** Input the number of channels of your images (grayscale = 1, RGB = 3). **Default value: 3**

* **`optimizer`:** Select the optimizer used to train your model. Options: ADAM, Stocastic Gradient Descent (SGD). ADAM converges usually faster but SGD is known for better generalization. **Default value: SGD**

* **`initial_learning_rate`:** Input the initial value to be used as learning rate. If you select ADAM as optimizer, this value should be around 10e-4. **Default value: 0.001**

In [15]:
#@markdown ###Name of the model:
model_name = "my_2d_classification_reduce" #@param {type:"string"}

#@markdown ### Data management:
validation_from_train = True #@param {type:"boolean"}
percentage_validation =  10 #@param {type:"number"}
test_ground_truth = False #@param {type:"boolean"}

#@markdown ### Basic training parameters:
number_of_classes = 75#@param {type:"number"}
number_of_epochs =  100#@param {type:"number"}
patience =  20#@param {type:"number"}

#@markdown ### Advanced training parameters:

model_architecture = "simple_cnn" #@param ["EfficientNetB0", "simple_cnn"]

batch_size =  32#@param {type:"number"}
patch_size = 224 #@param {type:"number"}

input_channels = 3 #@param {type:"number"}

optimizer = "ADAMW" #@param ["ADAM", "SGD", "ADAMW"]
initial_learning_rate = 0.001 #@param {type:"number"}

In [18]:
#@markdown ##Play to download the YAML configuration file and update it to train the model


os.chdir('/content/')

job_name = model_name
yaml_file = "/content/"+str(job_name)+".yaml"

# remove previous configuration file if it exists with the same name
if os.path.exists( yaml_file ):
    os.remove( yaml_file )

# Download template file
import shutil
shutil.copy("/content/BiaPy/templates/classification/classification_2d.yaml", yaml_file)

# Check folders before modifying the .yaml file
if not os.path.exists(train_data_path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), train_data_path)
ids = sorted(next(os.walk(train_data_path))[1])
if len(ids) == 0:
    raise ValueError("No folders found in dir {}".format(train_data_path))

if not validation_from_train:
    if not os.path.exists(val_data_path):
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), val_data_path)
    ids = sorted(next(os.walk(val_data_path))[1])
    if len(ids) == 0:
        raise ValueError("No folders found in dir {}".format(val_data_path))

if not os.path.exists(test_data_path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), test_data_path)
ids = sorted(next(os.walk(test_data_path))[1])
if len(ids) == 0:
    raise ValueError("No folders found in dir {}".format(test_data_path))


# open template configuration file
import yaml
with open( yaml_file, 'r') as stream:
    try:
        biapy_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# update paths to data
biapy_config['DATA']['TRAIN']['PATH'] = train_data_path
if validation_from_train:
    biapy_config['DATA']['VAL']['PATH'] = train_data_path
else:
    biapy_config['DATA']['VAL']['PATH'] = val_data_path
biapy_config['DATA']['TEST']['PATH'] = test_data_path
biapy_config['DATA']['TEST']['LOAD_GT'] = test_ground_truth

# update data patch size
biapy_config['DATA']['PATCH_SIZE'] = '('+str(patch_size)+', '+ str(patch_size)+', ' + str(input_channels)+')'
# adjust test padding accordingly
padding = patch_size // 8
biapy_config['DATA']['TEST']['PADDING'] = '('+str(padding)+', '+ str(padding)+')'

# update training parameters
biapy_config['DATA']['VAL']['FROM_TRAIN'] = validation_from_train
biapy_config['DATA']['VAL']['SPLIT_TRAIN'] = percentage_validation/100.0
biapy_config['TRAIN']['EPOCHS'] = number_of_epochs
biapy_config['TRAIN']['PATIENCE'] = patience
biapy_config['TRAIN']['BATCH_SIZE'] = batch_size
biapy_config['TRAIN']['OPTIMIZER'] = optimizer
biapy_config['TRAIN']['LR'] = initial_learning_rate

# Transcribe model architecture
# Available models: "simple_cnn", "EfficientNetB0"
architecture = 'simple_cnn'
if model_architecture == "simple_cnn":
    architecture = 'simple_cnn'
else:
    architecture = 'EfficientNetB0'
biapy_config['MODEL']['N_CLASSES'] = number_of_classes
biapy_config['MODEL']['ARCHITECTURE'] = architecture

# save file
with open( yaml_file, 'w') as outfile:
    yaml.dump(biapy_config, outfile, default_flow_style=False)

print( "Training configuration finished.")

Training configuration finished.


In [22]:
!ls /content/data/train | wc -l

75


In [23]:
#@markdown ##Play to train the model
# Run the code
os.chdir('/content/BiaPy')
!python -u main.py --config '/content/'{job_name}'.yaml' --result_dir {output_path} --name {job_name} --run_id 1 --gpu 0

Date: 2023-07-28 07:10:57
Arguments: Namespace(config='/content/my_2d_classification_reduce.yaml', result_dir='/content/output', name='my_2d_classification_reduce', run_id=1, gpu='0')
Job: my_2d_classification_reduce_1
Python       : 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]
Keras        : 2.12.0
Tensorflow   : 2.12.0
Num GPUs Available:  1
Configuration details:
AUGMENTOR:
  AFFINE_MODE: constant
  AUG_NUM_SAMPLES: 10
  AUG_SAMPLES: True
  BRIGHTNESS: False
  BRIGHTNESS_EM: False
  BRIGHTNESS_EM_FACTOR: (-0.1, 0.1)
  BRIGHTNESS_EM_MODE: 3D
  BRIGHTNESS_FACTOR: (-0.1, 0.1)
  BRIGHTNESS_MODE: 3D
  CBLUR_DOWN_RANGE: (2, 8)
  CBLUR_INSIDE: True
  CBLUR_SIZE: (0.2, 0.4)
  CHANNEL_SHUFFLE: False
  CMIX_SIZE: (0.2, 0.4)
  CNOISE_NB_ITERATIONS: (1, 3)
  CNOISE_SCALE: (0.05, 0.1)
  CNOISE_SIZE: (0.2, 0.4)
  CONTRAST: False
  CONTRAST_EM: False
  CONTRAST_EM_FACTOR: (-0.1, 0.1)
  CONTRAST_EM_MODE: 3D
  CONTRAST_FACTOR: (-0.1, 0.1)
  CONTRAST_MODE: 3D
  COUT_APPLY_TO_MASK: False
  COUT_C

In [ ]:
#@markdown ##Play to train the model
# Run the code
os.chdir('/content/BiaPy')
!python -u main.py --config '/content/'{job_name}'.yaml' --result_dir {output_path} --name {job_name} --run_id 1 --gpu 0

Date: 2023-07-27 13:08:21
Arguments: Namespace(config='/content/my_2d_classification_reduce.yaml', result_dir='/content/output', name='my_2d_classification_reduce', run_id=1, gpu='0')
Job: my_2d_classification_reduce_1
Python       : 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]
Keras        : 2.12.0
Tensorflow   : 2.12.0
Num GPUs Available:  1
Configuration details:
AUGMENTOR:
  AFFINE_MODE: constant
  AUG_NUM_SAMPLES: 10
  AUG_SAMPLES: True
  BRIGHTNESS: False
  BRIGHTNESS_EM: False
  BRIGHTNESS_EM_FACTOR: (-0.1, 0.1)
  BRIGHTNESS_EM_MODE: 3D
  BRIGHTNESS_FACTOR: (-0.1, 0.1)
  BRIGHTNESS_MODE: 3D
  CBLUR_DOWN_RANGE: (2, 8)
  CBLUR_INSIDE: True
  CBLUR_SIZE: (0.2, 0.4)
  CHANNEL_SHUFFLE: False
  CMIX_SIZE: (0.2, 0.4)
  CNOISE_NB_ITERATIONS: (1, 3)
  CNOISE_SCALE: (0.05, 0.1)
  CNOISE_SIZE: (0.2, 0.4)
  CONTRAST: False
  CONTRAST_EM: False
  CONTRAST_EM_FACTOR: (-0.1, 0.1)
  CONTRAST_EM_MODE: 3D
  CONTRAST_FACTOR: (-0.1, 0.1)
  CONTRAST_MODE: 3D
  COUT_APPLY_TO_MASK: False
  COUT_C